## Cargar matrices

In [3]:
using MAT
using ReachabilityAnalysis
using Plots
using SparseArrays
using LinearAlgebra

LazySets.set_ztol(Float64, 1e-30)

1.0e-30

In [121]:
d = matread("ONSAS_call/output/matrices.mat")

K = d["KTred"]
M = d["massMatred"];
neumdofs = d["neumdofs"];
neumdofs = Int.(neumdofs)[:];

In [10]:
m = size(K, 1)

1186

In [11]:
extrema(K)

(-1.9950820594914427e9, 3.925875723970502e9)

In [12]:
extrema(M)

(0.0, 0.023502432487921353)

## Definir fuerza externa

In [161]:
df = matread("ONSAS_call/output/loads.mat")
F = df["factorLoadsFextCell"][1, 3][:]
F = F[neumdofs];
Fnotzerocoord = findall(!iszero, F);

In [126]:
Heaviside(t) = t >= 0 ? 1 : 0

# funcion auxiliar que devuelve el factor de carga aplico en la coordenada coord
function factor_de_carga(m, NSTEPS; T=1.0, coord=5, func=Heaviside)
    tiempos = range(0.0, T, length=NSTEPS)
    
    F = Vector{Vector{Float64}}(undef, NSTEPS)
    for i in 1:NSTEPS
        aux = zeros(m)
        aux[coord] .= func(tiempos[i])
        
        F[i] = aux
    end
    return F
end

factor_de_carga (generic function with 1 method)

In [127]:
H = Heaviside

fHeav(t) = -1e3 * H(t)

tsp = range(-1e-2, 1.0, length=100)
plot(tsp, t -> fHeav.(t), xlab="t", ylab="x(t)", lw=2.0, lab="", title="External force: Heaviside")

<!DOCTYPE html>
 
 
 Plots.jl

$$
F_{rickler}(t) = 5\times10^6 \left( H(0.00015-t) - 3H(0.0001-t) + 3H(0.00005-t) \right)
$$

In [198]:
ancho_escalon = 20e-6

fr(t) = 5e6 * (H(ancho_escalon*3 - t) - 3*H(ancho_escalon*2 - t) + 3*H(ancho_escalon - t))

# horizonte maximo: 0.5ms
tsp = range(0.0, 0.5e-3, length=100)
plot(tsp, t -> fr.(t), xlab="t", ylab="x(t)", lw=2.0, lab="", title="External force")

<!DOCTYPE html>
 
 
 Plots.jl

## Solucion con los metodos numericos

In [129]:
using StructuralDynamicsODESolvers

In [130]:
# plotly() # descomentar si esta instalado, para los zoom

Plots.PlotlyBackend()

In [199]:
#F = zeros(m); F[5] = -1e6
#R_heav = factor_de_carga(m, 1001; T=1e-3, coord=Fnotzerocoord, func=t -> -1e6 * H(t));
# factor_de_carga(m, 1001; T=1e-3, coord=Fnotzerocoord, func=fr);

NSTEPS = 100
Tmax = 0.5e-3

tsp = range(0.0, Tmax, length=NSTEPS+1)
R_ricker = [F * fr(ti) for ti in tsp];

In [200]:
C = spzeros(m, m)

# caso F constante
# sys = SecondOrderAffineContinuousSystem(M, C, K, F);

# caso F variable
#sys = SecondOrderConstrainedLinearControlContinuousSystem(M, C, K, Matrix(1.0I, m, m), Universe(m), R_heav)
sys = SecondOrderConstrainedLinearControlContinuousSystem(M, C, K, Matrix(1.0I, m, m), Universe(m), R_ricker)

prob = @ivp(sys, u(0) ∈ (zeros(m), zeros(m)))

# simulation step size
δn = 5e-6

alg = Trapezoidal(Δt=δn)

@time sol = solve(prob, alg, NSTEPS=NSTEPS);

  0.136784 seconds (2.62 k allocations: 29.961 MiB)


- historia de nodos 13 y 23 a comparar

- dofs: 13*2-1 13*2 23*2-1 23*2 a esto se le resta 4 por los dos nodos fijos

- dofs: 13x2-1 13x2 23x2-1 23x2 a esto se le resta 4 por los dos nodos fijos

- nodos a chequear comportamiento esperado: 4 y 5 deberían ser casi iguales 4x2 5x2

In [201]:
fig = plot(xlab="Time (s)", ylab="Vertical displacement (m)")
plot!(fig, sol, vars=(0, 4*2-2), lab="Node 4")
plot!(fig, sol, vars=(0, 5*2-2), lab="Node 5")
plot!(fig, tsp, t -> 1e-10 * fr.(t), lw=3.0, c=:black, ls=:dash, lab="Force")

<!DOCTYPE html>
 
 
 Plots.jl

In [202]:
fig = plot(xlab="Time (s)", ylab="Vertical displacement (m)")
plot!(fig, sol, vars=(0, 23*2-4), lab="Node 23")
plot!(fig, sol, vars=(0, 13*2-4), lab="Node 13")
plot!(fig, tsp, t -> 1e-10 * fr.(t), lw=3.0, c=:black, ls=:dash, lab="Force")

<!DOCTYPE html>
 
 
 Plots.jl

## Plot en x-y

In [139]:
mesh = matread("ONSAS_call/mesh_dos_puntos.mat")
coords = mesh["mesh"]["nodesCoords"]
connec = mesh["mesh"]["conecCell"];

In [140]:
size(coords)

(595, 3)

In [141]:
xcoords = coords[:, 1]
ycoords = coords[:, 2]
undeformed = UnionSetArray([Singleton([x, y]) for (x, y) in zip(xcoords, ycoords)]);

In [147]:
ycoords[4]

0.04000000000000001

In [150]:
ycoords[23]

0.0

In [156]:
fig = plot()

plot!(fig, undeformed, ratio=1.)
plot!(fig, undeformed.array[13], c=:red, lab="13")
plot!(fig, undeformed.array[23], c=:orange, lab="23")
plot!(fig, undeformed.array[4], c=:yellow, lab="4")
plot!(fig, undeformed.array[5], c=:magenta, lab="5")

<!DOCTYPE html>
 
 
 Plots.jl

In [ ]:
#=
# dof libres
dof = collect(1:1056)
deleteat!(dof, [1, 2, 4]);

Ug = [zeros(1056) for _ in 1:length(sol.U)]
for i in 1:length(sol.U)
    Ug[i][dof] .= sol.U[i]
end

deformed = []
for k in 1:length(sol.U)
    xdispl = Ug[k][1:2:1056] + xcoords
    ydispl = Ug[k][2:2:1056] + ycoords
    push!(deformed, UnionSetArray([Singleton([x, y]) for (x, y) in zip(xdispl, ydispl)]))
end

fig = plot(xlims=(0.0, 0.1), ylims=(-0.01, 0.1))
@gif for i ∈ 1:100
    plot!(fig, deformed[i], c=:red)
end every 10

h = box_approximation(ConvexHullArray([box_approximation(deformed[i]) for i in 1:length(deformed)]))

a = h.center - h.radius

b = h.center + h.radius

for i in 1:length(deformed)
    fig = plot(xlims=(a[1], b[1]), ylims=(a[2], b[2]))
    x = @sprintf("%05i", i)
    plot!(fig, deformed[i], c=:red)

    savefig(fig, "fig/deformed_$x.png")
end
=#

## Homogeneizar

In [157]:
# homogeneization
m = size(M, 1)

1186

In [158]:
Zm = zeros(m, m)
Im = Matrix(1.0I, m, m)
invM = inv(Matrix(M))

A = [Zm           Im;
    -invM*K       Zm];

In [162]:
# MX'' + KX = F
F0 = vcat(zeros(m), invM * F)

n = 2m
Aext = zeros(n+1, n+1)
Aext[1:n, 1:n] .= A
Aext[1:n, n+1] .= F0
Aext = sparse(Aext)

S = @system(x' = Aext*x); # sistema homogeneizado

## Calculo preciso

In [177]:
tsp = range(0.0, Tmax, length=NSTEPS+1)
#R_ricker = [F * fr(ti) for ti in tsp];
#fr(tsp[1])
fr(5e-5 + 1e-12)

-1.0e7

In [194]:
# ---------------
# PRIMER TRAMO
# ---------------

δt = δn # reachability step size
X0 = zeros(2m + 1)
X0[end] = fr(0.0)
X0 = ReachSet(Singleton(X0), 0 .. 0)

ivp = @ivp(S, x(0) ∈ X0);
@time Φ = exp(Matrix(state_matrix(ivp)) * δt); # Φ = exp(Aδ)
@show N1 = round(Int, 5e-5 / δt)
ΦN1 = Φ^N1
X02 = ReachSet(linear_map(ΦN1, set(X0)), interval(5e-5));

  3.485678 seconds (44 allocations: 816.319 MiB, 8.12% gc time)
N1 = round(Int, 5.0e-5 / δt) = 10


In [ ]:
# ---------------
# SEGUNDO TRAMO
# ---------------
X02.X.element[end] = fr(5e-5 + 1e-12);
X03 = ReachSet(linear_map(Φ128, set(X02)), interval(0.1));

# ---------------
# TERCER TRAMO
# ---------------
X03.X.element[end] = 2e6;
X04 = ReachSet(linear_map(Φ128, set(X03)), interval(0.15));

In [193]:
fig = plot(xlab="Time (s)", ylab="Vertical displacement (m)")
plot!(fig, sol, vars=(0, 4*2-2), lab="Node 4")
plot!(fig, sol, vars=(0, 5*2-2), lab="Node 5")
plot!(fig, tsp, t -> 1e-10 * fr.(t), lw=3.0, c=:black, ls=:dash, lab="Force")

plot!(fig, X0, vars=(0, 4*2-2), lab="X0", c=:magenta)
plot!(fig, X02, vars=(0, 4*2-2), lab="X02", c=:magenta)

#plot!(X03, vars=(0, 117), c=:blue)
#plot!(X04, vars=(0, 117), c=:blue)
#plot!(sol4, vars=(0, 117), c=:blue)

<!DOCTYPE html>
 
 
 Plots.jl

## Tramo pesado

In [ ]:
# ---------------
# CUARTO TRAMO
# ---------------

#X04.X.element[end] = 0.0;
#alg = BOX(δ=dtn/20., approx_model=Forward(setops=:box))
#@time sol4 = solve(IVP(S, set(X04)), alg=alg, T=3-0.15, Δt0=interval(0.15));